## RAG 기반 Chatbot

In [1]:
# 1. 필요한 라이브러리 import (모든 import를 상단에 모음)
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # 최신 import 방식
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
# 2. 환경 변수 로드 및 API 키 설정
load_dotenv()  # .env 파일에서 환경변수 로드
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# 3. PDF 문서 로드
pdf_filepath = './data/소나기.pdf'
loader = ...  # ... code here
data = ...    
print(f"PDF 로드 완료: {len(data)}개 페이지")# ... code here

PDF 로드 완료: 7개 페이지


In [ ]:
# 4. 텍스트 분할 (청킹)
text_splitter = ... (  # ... code here
    chunk_size=1000,    # 청크 크기: 1000자
    chunk_overlap=200   # 청크 간 겹침: 200자
)
data = ...(data)  # ... code here
print(f"텍스트 분할 완료: {len(data)}개 청크")

텍스트 분할 완료: 7개 청크


In [ ]:
# 5. OpenAI 임베딩 모델 초기화
embeddings = ... (  # ... code here
    model="text-embedding-ada-002", 
    api_key=api_key
)

In [ ]:
# 6. FAISS 벡터스토어 생성 (문서를 벡터로 변환하여 저장) : Document 객체 리스트 입력 (chunk된 거)
vectorstore = ... (data, embeddings)  # ... code here

In [10]:
# 7. ChatGPT 모델 초기화
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    api_key=api_key,
    temperature=0  # 일관된 답변을 위해 추가
)

In [ ]:
# 8. 대화 메모리 설정 (이전 대화 내용 기억)
memory = ... (  # ... code here
    memory_key='chat_history',  # 메모리 키 설정
    return_messages=True        # 메시지 형태로 반환
)

In [ ]:
# 9. 대화형 검색 체인 생성 (RAG + 대화 기능) - ConversationalRetrievalChain 사용
conversation_chain = ... (  # ... code here
    llm=llm,                                    # 사용할 LLM 모델
    chain_type="...",      # ... code here      # 문서 처리 방식 : 문서를 한꺼번에 넘기는 방식
    retriever=...,        # ... code here       # 벡터 검색기
    memory=memory                               # 대화 메모리
)

In [ ]:
# 10. 첫 번째 질문 실행
query1 = "소년은 어디에서 처음 소녀를 만났나?"
print(f"\n질문 1: {query1}")

result1 = ... ({"question": query1})    # ... code here 
answer1 = result1["answer"]
print(f"답변 1: {answer1}")


질문 1: 소년은 어디에서 처음 소녀를 만났나?
답변 1: 소년은 처음으로 소녀를 개울가에서 만났습니다.


In [14]:
# 11. 두 번째 질문 실행 (이전 대화 맥락 유지)
query2 = "소년은 소녀에게 무엇을 주려고 했나?"
print(f"\n질문 2: {query2}")

result2 = conversation_chain.invoke({"question": query2})  # invoke 사용
answer2 = result2["answer"]
print(f"답변 2: {answer2}")


질문 2: 소년은 소녀에게 무엇을 주려고 했나?
답변 2: 소년은 소녀에게 대추를 주려고 했습니다.


In [ ]:
# 질의응답 함수로 만들기 (재사용 가능)
def ask_question(chain, question):
    """질문을 하고 답변을 받는 함수"""
    print(f"\n 질문: {question}")
    result = chain.invoke({"question": question})  # 최신 방식
    answer = result["answer"]
    print(f" 답변: {answer}")
    return answer

# 질문들 실행
questions = [
    "소년은 어디에서 처음 소녀를 만났나?",
    "소년은 소녀에게 무엇을 주려고 했나?", 
    "그 장소는 어떤 특징이 있었나?",
    "소나기는 언제 내렸나?"
]

for i, question in enumerate(questions, 1):
    answer = ask_question(conversation_chain, question)
    if i < len(questions):
        print("-" * 50)


 질문: 소년은 어디에서 처음 소녀를 만났나?
 답변: 소년은 처음으로 소녀를 개울가에서 만났습니다.
----------------------------------------

 질문: 소년은 소녀에게 무엇을 주려고 했나?
 답변: 소년은 소녀에게 대추를 주려고 했습니다.
----------------------------------------

 질문: 그 장소는 어떤 특징이 있었나?
 답변: 해당 장면은 개울가와 징검다리가 중심에 있었던 것으로 보입니다. 소년과 소녀가 만나고 대화를 나누는 장면에서 주변 환경과 자연의 아름다움이 묘사되었습니다. 또한, 주변에는 갈밭과 메밀밭, 허수아비 등의 풍경이 묘사되어 있습니다.
----------------------------------------

 질문: 소나기는 언제 내렸나?
 답변: 소나기는 주어진 텍스트에서 언급되지 않았습니다. 따라서 소나기가 언제 내렸는지에 대한 정보는 제공할 수 없습니다.
